# Muscle Fatigue Detection - Example Notebook

This notebook demonstrates how to use the muscle fatigue detection system to classify EMG signals.

In [ ]:
import numpy as np
import pandas as pd
import sys
import os

# Add src to path
sys.path.insert(0, os.path.join(os.getcwd(), '..', 'src'))

from src.preprocessing import EMGPreprocessor
from src.feature_extraction import EMGFeatureExtractor
from src.models import FatigueClassifier
from src.pipeline import FatigueDetectionPipeline, train_multiple_models
from src.visualization import (
    plot_emg_signal,
    plot_confusion_matrix,
    plot_model_comparison,
    plot_feature_distributions
)
from sklearn.model_selection import train_test_split

%matplotlib inline

## 1. Generate Synthetic EMG Data

For demonstration, we'll generate synthetic EMG signals simulating fatigued and non-fatigued states.

In [ ]:
def generate_synthetic_emg(n_samples=50, signal_length=5000, sampling_rate=1000):
    """Generate synthetic EMG data."""
    signals = []
    labels = []
    
    for i in range(n_samples):
        is_fatigued = i % 2
        t = np.arange(signal_length) / sampling_rate
        
        if is_fatigued:
            freq = np.random.uniform(15, 30)
            amplitude = np.random.uniform(0.8, 1.5)
            noise_level = 0.3
        else:
            freq = np.random.uniform(50, 100)
            amplitude = np.random.uniform(0.3, 0.7)
            noise_level = 0.2
        
        signal = amplitude * np.sin(2 * np.pi * freq * t)
        signal += 0.3 * amplitude * np.sin(2 * np.pi * freq * 2 * t)
        signal += np.random.normal(0, noise_level, signal_length)
        
        signals.append(signal)
        labels.append(is_fatigued)
    
    return signals, labels

signals, labels = generate_synthetic_emg(n_samples=100)
print(f"Generated {len(signals)} signals")
print(f"Non-fatigued: {labels.count(0)}, Fatigued: {labels.count(1)}")

## 2. Visualize Sample Signals

In [ ]:
# Plot a non-fatigued signal
plot_emg_signal(signals[0], sampling_rate=1000, title="Non-Fatigued EMG Signal")

In [ ]:
# Plot a fatigued signal
plot_emg_signal(signals[1], sampling_rate=1000, title="Fatigued EMG Signal")

## 3. Preprocess and Extract Features

In [ ]:
# Create pipeline
pipeline = FatigueDetectionPipeline(sampling_rate=1000, model_type='knn')

# Prepare dataset
features_df, labels_array = pipeline.prepare_dataset(
    signals, labels, window_size=1000, overlap=0.5
)

print(f"Extracted {len(features_df)} feature vectors")
print(f"Features: {list(features_df.columns)}")
print(f"\nFeature statistics:")
print(features_df.describe())

## 4. Visualize Feature Distributions

In [ ]:
plot_feature_distributions(
    features_df, 
    labels_array, 
    feature_names=['rms', 'median_frequency', 'mean_frequency', 'spectral_entropy']
)

## 5. Split Data and Train Models

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    features_df, labels_array, test_size=0.2, random_state=42, stratify=labels_array
)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

In [ ]:
# Train multiple models
results = train_multiple_models(X_train, y_train, X_test, y_test)

## 6. Visualize Results

In [ ]:
# Compare model accuracy
plot_model_comparison(results)

In [ ]:
# Plot confusion matrix for best model
best_model_name = max(results.items(), key=lambda x: x[1]['metrics']['accuracy'])[0]
best_result = results[best_model_name]

plot_confusion_matrix(
    best_result['metrics']['confusion_matrix'],
    class_names=['Non-fatigued', 'Fatigued'],
    title=f"{best_model_name.upper()} Confusion Matrix"
)

## 7. Test Prediction on New Signal

In [ ]:
# Generate a test signal
test_signals, test_labels = generate_synthetic_emg(n_samples=1)
test_signal = test_signals[0]
true_label = test_labels[0]

# Make prediction
test_pipeline = FatigueDetectionPipeline(sampling_rate=1000, model_type=best_model_name)
test_pipeline.classifier = best_result['classifier']

predictions = test_pipeline.predict(test_signal, window_size=1000, overlap=0.5)
predicted_label = int(np.round(np.mean(predictions)))

print(f"True label: {'Fatigued' if true_label == 1 else 'Non-fatigued'}")
print(f"Predicted label: {'Fatigued' if predicted_label == 1 else 'Non-fatigued'}")
print(f"Prediction confidence: {np.mean(predictions):.2f}")

## 8. Save Best Model

In [ ]:
# Save model
model_path = f'../models/notebook_model_{best_model_name}.joblib'
best_result['classifier'].save_model(model_path)
print(f"Model saved to: {model_path}")

## Next Steps

1. Replace synthetic data with real EMG dataset
2. Tune hyperparameters for better performance
3. Implement cross-validation
4. Analyze feature importance
5. Deploy model for real-time fatigue detection